# rgb-lib issuance and transfer demo
Example of asset issuance and transfer using [rgb-lib](https://github.com/RGB-Tools/rgb-lib) via its [Python bindings](https://github.com/RGB-Tools/rgb-lib-python)

### imports

In [1]:
import os
import shutil

#import magic
import matplotlib.pyplot as plot
import qrcode
from IPython.display import Image, display

import rgb_lib

OSError: dlopen(/Users/walter/Library/Caches/pypoetry/virtualenvs/rgbassets-JE_iEHuL-py3.10/lib/python3.10/site-packages/rgb_lib/_rgb_lib/librgblibffi.dylib, 0x0006): tried: '/Users/walter/Library/Caches/pypoetry/virtualenvs/rgbassets-JE_iEHuL-py3.10/lib/python3.10/site-packages/rgb_lib/_rgb_lib/librgblibffi.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/walter/Library/Caches/pypoetry/virtualenvs/rgbassets-JE_iEHuL-py3.10/lib/python3.10/site-packages/rgb_lib/_rgb_lib/librgblibffi.dylib' (no such file), '/Users/walter/Library/Caches/pypoetry/virtualenvs/rgbassets-JE_iEHuL-py3.10/lib/python3.10/site-packages/rgb_lib/_rgb_lib/librgblibffi.dylib' (no such file)

### common values

In [ ]:
electrum_url = 'tcp://electrs:50001'
proxy_url = 'https://proxy.rgbtools.org'
bitcoin_network = rgb_lib.BitcoinNetwork.REGTEST

---
# receiver
---

### generate a new wallet

In [ ]:
recv_keys = rgb_lib.generate_keys(bitcoin_network)
print('receiver wallet keys:')
print(' - mnemonic:', recv_keys.mnemonic)
print(' - xpub:', recv_keys.xpub)

### initialize the wallet

In [ ]:
recv_data_dir = './data/recv_wallet'
recv_mnemonic = recv_keys.mnemonic
recv_xpub = recv_keys.xpub

if not os.path.exists(recv_data_dir):
    os.makedirs(recv_data_dir)

recv_wallet_data = rgb_lib.WalletData(
        recv_data_dir,
        bitcoin_network,
        rgb_lib.DatabaseType.SQLITE,
        recv_xpub,
        recv_mnemonic,
)

recv_wallet = rgb_lib.Wallet(recv_wallet_data)
recv_online = recv_wallet.go_online(False, electrum_url, proxy_url)
print(f'wallet initialized, fingerprint: {recv_keys.xpub_fingerprint}')

### get a Bitcoin address to add some funds
fund the address with some regtest bitcoins
```shell
./services.sh fund <address>
```

### create UTXOs to hold RGB allocations

In [ ]:
recv_address = recv_wallet.get_address()

recv_image = qrcode.make(recv_address)
plot.imshow(recv_image,cmap='gray')
print(recv_address)

In [ ]:
recv_wallet.create_utxos(recv_online, True, 5)

### check wallet unspents

In [ ]:
recv_wallet.refresh(recv_online, None)
recv_unspents = recv_wallet.list_unspents(settled_only=False)
for unspent in recv_unspents:
    print(unspent.utxo)
    for allocation in unspent.rgb_allocations:
        print(f'\t- {allocation}')

### prepare blinded UTXOs to receive assets

In [ ]:
recv_blind_data_1 = recv_wallet.blind(None, None)
recv_blind_data_2 = recv_wallet.blind(None, None)
print(f'blinded UTXO 1: {recv_blind_data_1.blinded_utxo}')
print(f'blinded UTXO 2: {recv_blind_data_2.blinded_utxo}')

---
# sender
---

### generate a new wallet
each time `rgb_lib.generate_keys()` is called a new mnemonic and xpub are created, giving a brand new wallet

In [ ]:
send_keys = rgb_lib.generate_keys(bitcoin_network)
print('new wallet keys:')
print(' - mnemonic:', send_keys.mnemonic)
print(' - xpub:', send_keys.xpub)

### initialize the wallet
copy the generated mnemonic and xpub and set them to their respective variables below,
this way the bitcoin wallet won't change unexpectedly between runs

In [ ]:
send_data_dir = './data/send_wallet'
send_mnemonic = send_keys.mnemonic
send_xpub = send_keys.xpub

if not os.path.exists(send_data_dir):
    os.makedirs(send_data_dir)

send_wallet_data = rgb_lib.WalletData(
        send_data_dir,
        bitcoin_network,
        rgb_lib.DatabaseType.SQLITE,
        send_xpub,
        send_mnemonic,
)

send_wallet = rgb_lib.Wallet(send_wallet_data)
send_online = send_wallet.go_online(False, electrum_url, proxy_url)
print(f'wallet initialized, fingerprint: {send_keys.xpub_fingerprint}')

### get a Bitcoin address to add some funds
fund the address with some regtest bitcoins
```shell
./services.sh fund <address>
```

In [ ]:
send_address = send_wallet.get_address()
print(send_address)

send_image = qrcode.make(send_address)
plot.imshow(send_image,cmap='gray')

### create UTXOs to hold RGB allocations

In [ ]:
send_wallet.create_utxos(send_online, True, 5)

### check wallet unspents

In [ ]:
send_wallet.refresh(send_online, None)
send_unspents = send_wallet.list_unspents(settled_only=False)
for unspent in send_unspents:
    print(unspent.utxo)
    for allocation in unspent.rgb_allocations:
        print(f'\t- {allocation}')

### issue an RGB20 asset

In [ ]:
# asset data
name = 'Jupyter Fungible Token'
precision = 0
amounts = [1000]
ticker = 'JFT'

# issue the asset
rgb20_asset = send_wallet.issue_asset_rgb20(send_online, ticker, name, precision, amounts)
print(f'issued asset with ID: {rgb20_asset.asset_id}')

### issue an RGB21 asset
place an PNG image in the demo directory, named `sample.png`

In [ ]:
# asset data
name = 'JupyNFT'
precision = 0
amounts = [1]
description = 'Jupyter Collectible'
parent_id = None
file_path = 'sample.png'

if not os.path.exists(file_path):
    raise RuntimeError(f'missing image file: {file_path}')

# issue the asset
rgb21_asset = send_wallet.issue_asset_rgb21(
    send_online, name, description, precision, amounts, parent_id, file_path)
print(f'issued asset with ID: {rgb21_asset.asset_id}')

### list wallet assets

In [ ]:
assets = send_wallet.list_assets(filter_asset_types=[])

print('RGB20 assets:')
for asset in assets.rgb20:
    print(f'- {asset}')
    
print('\nRGB21 assets:')
for asset in assets.rgb21:
    print(f'- {asset}')

### send some assets to the receiver's blinded UTXO
- create PSBT and RGB transfer
- anchor RGB transfer to PSBT
- post consignment to proxy server

#### RGB20

In [ ]:
amount_rgb20 = 10

recipient_map_rgb20 = {
    rgb20_asset.asset_id: [
        rgb_lib.Recipient(recv_blind_data_1.blinded_utxo, amount_rgb20),
    ]
}
txid = send_wallet.send(send_online, recipient_map_rgb20, False)

print(f'RGB20 txid: {txid}')

#### RGB21

In [ ]:
amount_rgb21 = 1

recipient_map_rgb21 = {
    rgb21_asset.asset_id: [
        rgb_lib.Recipient(recv_blind_data_2.blinded_utxo, amount_rgb21),
    ]
}
txid = send_wallet.send(send_online, recipient_map_rgb21, False)

print(f'RGB21 txid: {txid}')

### list asset transfers

In [ ]:
print('RGB20 asset transfers:')
send_transfers = send_wallet.list_transfers(rgb20_asset.asset_id)
for transfer in send_transfers:
    print(f'- {transfer}')
    
print('\nRGB21 asset transfers:')
send_transfers = send_wallet.list_transfers(rgb21_asset.asset_id)
for transfer in send_transfers:
    print(f'- {transfer}')

---
# receiver
---

### refresh transfers
- check pending transfers for incoming consignments
- get the consignment from the proxy server and validate it
- ACK the transfer on the proxy server

In [ ]:
recv_wallet.refresh(recv_online, None)

### list asset transfers

In [ ]:
print('RGB20 asset transfers:')
recv_transfers = recv_wallet.list_transfers(rgb20_asset.asset_id)
for transfer in recv_transfers:
    print(f'- {transfer}')

print('\nRGB21 asset transfers:')
recv_transfers = recv_wallet.list_transfers(rgb21_asset.asset_id)
for transfer in recv_transfers:
    print(f'- {transfer}')

---
# sender
---

### refresh transfers
- check pending transfers for ACKs
- broadcast transaction for ACKed transfers

In [ ]:
send_wallet.refresh(send_online, None)

### list asset transfers

In [ ]:
print('RGB20 asset transfers:')
send_transfers = send_wallet.list_transfers(rgb20_asset.asset_id)
for transfer in send_transfers:
    print(f'- {transfer}')
    
print('\nRGB21 asset transfers:')
send_transfers = send_wallet.list_transfers(rgb21_asset.asset_id)
for transfer in send_transfers:
    print(f'- {transfer}')

---
### transaction confirmation
for the transfer to progress to a final state, mine a block
```shell
./services.sh mine 1
```

---
# receiver
---

### refresh transfers
- check pending transfers for transaction confirmations

In [ ]:
recv_wallet.refresh(recv_online, None)

### list asset transfers

In [ ]:
print('RGB20 asset transfers:')
recv_transfers = recv_wallet.list_transfers(rgb20_asset.asset_id)
for transfer in recv_transfers:
    print(f'- {transfer}')
    assert(transfer.status == rgb_lib.TransferStatus.SETTLED)


print('\nRGB21 asset transfers:')
recv_transfers = recv_wallet.list_transfers(rgb21_asset.asset_id)
for transfer in recv_transfers:
    print(f'- {transfer}')
    assert(transfer.status == rgb_lib.TransferStatus.SETTLED)

In [ ]:
recv_assets = recv_wallet.list_assets([])

print('RGB20 assets:')
for asset in recv_assets.rgb20:
    print(f'- {asset}')
    
print('\nRGB21 assets:')
magic_mime = magic.Magic(mime=True)
local_file_path = 'data/image.png'
for asset in recv_assets.rgb21:
    print(f'- {asset}')
    data_paths = asset.data_paths
    for media in data_paths:
        media_mime = media.mime
        media_path = media.file_path
        file_mime = magic_mime.from_file(media_path)
        assert(media_mime == 'image/png')
        assert(file_mime == 'image/png')
        shutil.copyfile(media_path, local_file_path) 
        display(Image(filename=local_file_path))